# The Battle of the Neighborhoods - Week 2 


This projects aims to find suitable location to open a new restaurant in NewYork city. Considering the highly competitive market, it is very important to startegically plan. In this section of the capstone project, we will use the Foursquare API to explore neighborhoods in Brooklyn, Manhattan, Bronx, Queens and Staten Island of NewYork City. We will use the explore function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. We will use the k-means clustering algorithm to complete this task. Finally, we will use the Folium library to visualize the neighborhoods different boroughs and their emerging clusters.


In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

# import k-means from clustering stage
from sklearn.cluster import KMeans

from sklearn.metrics import silhouette_score

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    cer

## Download and Explore Dataset


In [68]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


In [69]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Lets define a new variable that includes this data.

In [70]:
neighborhoods_data = newyork_data['features']

In [71]:
neighborhoods_data[0]# Look at first item in Data

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [72]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [73]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then loop through the data and fill the dataframe one row at a time.

In [74]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [75]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [76]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [77]:
 neighborhoods.to_csv('NYC_Geo.csv',index=False)
    

In [78]:
NYC_Geo = neighborhoods[(neighborhoods['Borough'] == 'Brooklyn')|(neighborhoods['Borough'] == 'Manhattan')
                        |(neighborhoods['Borough'] == 'Staten Island')
                        |(neighborhoods['Borough'] == 'Queens')
                        |(neighborhoods['Borough'] == 'Bronx')].reset_index(drop=True)
NYC_Geo.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Use geopy library to get the latitude and longitude values of New York City.

In [79]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="Jupyter")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [101]:
map_NewYork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NewYork)  
    
map_NewYork

In [99]:
map_NewYork.save('map_NewYork.html')

## Segmenting and Clustering Neighborhoods In All Boroughs of NEWYORK CITY

### Define Foursquare Credentials and Version

In [102]:
CLIENT_ID = 'DUXLBZ30P0KXSD54OSQP4HVTZKV444Z5TM2DFGPCIDAY2SUO' # your Foursquare ID
CLIENT_SECRET = 'JO32VVCAPM4OKY4TYXAUL2JGO1TESJE53EKZDDY0JQF1UQTQ' # your Foursquare Secret
VERSION = '20181218' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DUXLBZ30P0KXSD54OSQP4HVTZKV444Z5TM2DFGPCIDAY2SUO
CLIENT_SECRET:JO32VVCAPM4OKY4TYXAUL2JGO1TESJE53EKZDDY0JQF1UQTQ


## #Let's explore the first neighborhood in our dataframe.
Get the neighborhood's name.

In [81]:
NYC_Geo.loc[0, 'Neighborhood']

'Wakefield'

In [82]:
neighborhood_latitude = NYC_Geo.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = NYC_Geo.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = NYC_Geo.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Wakefield are 40.89470517661, -73.84720052054902.


In [83]:
def getNearbyVenues(names, latitudes, longitudes, LIMIT=200, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
NYC_venues = getNearbyVenues(names=NYC_Geo['Neighborhood'],
                                  latitudes=NYC_Geo['Latitude'],
                                  longitudes=NYC_Geo['Longitude'],
                                  LIMIT=200)

print('The "NYC_venues" dataframe has {} venues and {} unique venue types.'.format(len(NYC_venues['Venue Category']),
      len(NYC_venues['Venue Category'].unique())))


NYC_venues.head()

In [85]:
NYC_venues.groupby('Venue Category')['Venue'].count().sort_values(ascending=False)

Venue Category
Pizza Place                                 1009
Coffee Shop                                  545
Italian Restaurant                           543
Deli / Bodega                                468
Donut Shop                                   466
Bakery                                       437
Sandwich Place                               429
Park                                         414
Chinese Restaurant                           413
Bar                                          405
Pharmacy                                     399
Grocery Store                                349
Ice Cream Shop                               334
Bank                                         333
Mexican Restaurant                           322
American Restaurant                          321
Café                                         319
Supermarket                                  303
Fast Food Restaurant                         277
Gym                                          257
Carib

In [86]:
NYC_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,70,70,70,70,70,70
Annadale,17,17,17,17,17,17
Arden Heights,25,25,25,25,25,25
Arlington,20,20,20,20,20,20
Arrochar,24,24,24,24,24,24
Arverne,32,32,32,32,32,32
Astoria,100,100,100,100,100,100
Astoria Heights,74,74,74,74,74,74
Auburndale,100,100,100,100,100,100


In [87]:
print('There are {} uniques categories.'.format(len(NYC_venues['Venue Category'].unique())))

There are 478 uniques categories.


## Analyze Each Neighborhood

In [88]:
# one hot encoding
NYC_onehot = pd.get_dummies(NYC_venues[['Venue Category']], prefix="", prefix_sep="")

#column lists before adding neighborhood
column_names = ['Neighborhood'] + list(NYC_onehot.columns)

# add neighborhood column back to dataframe
NYC_onehot['Neighborhood'] = NYC_venues['Neighborhood'] 

# move neighborhood column to the first column
NYC_onehot = NYC_onehot[column_names]

NYC_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Center,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Gym,College Quad,College Residence Hall,College Theater,College Track,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cruise,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Kofte Place,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern

In [89]:
restaurant_List = []
search = 'Restaurant'
for i in NYC_onehot.columns :
    if search in i:
        restaurant_List.append(i)

In [71]:
restaurant_List

['Afghan Restaurant',
 'African Restaurant',
 'American Restaurant',
 'Arepa Restaurant',
 'Argentinian Restaurant',
 'Asian Restaurant',
 'Australian Restaurant',
 'Austrian Restaurant',
 'Brazilian Restaurant',
 'Burmese Restaurant',
 'Cajun / Creole Restaurant',
 'Cambodian Restaurant',
 'Cantonese Restaurant',
 'Caribbean Restaurant',
 'Caucasian Restaurant',
 'Chinese Restaurant',
 'Colombian Restaurant',
 'Comfort Food Restaurant',
 'Cuban Restaurant',
 'Czech Restaurant',
 'Dim Sum Restaurant',
 'Dumpling Restaurant',
 'Eastern European Restaurant',
 'Egyptian Restaurant',
 'Empanada Restaurant',
 'English Restaurant',
 'Ethiopian Restaurant',
 'Falafel Restaurant',
 'Fast Food Restaurant',
 'Filipino Restaurant',
 'French Restaurant',
 'German Restaurant',
 'Gluten-free Restaurant',
 'Greek Restaurant',
 'Halal Restaurant',
 'Hawaiian Restaurant',
 'Himalayan Restaurant',
 'Hotpot Restaurant',
 'Indian Restaurant',
 'Indonesian Restaurant',
 'Israeli Restaurant',
 'Italian Rest

In [90]:
col_name = []
col_name = ['Neighborhood'] + restaurant_List
NYC_restaurant = NYC_onehot[col_name]
NYC_restaurant = NYC_restaurant.iloc[:,1::]

In [91]:
NYC_restaurant_grouped = NYC_restaurant.groupby('Neighborhood').sum().reset_index()
NYC_restaurant_grouped['Total'] = NYC_restaurant_grouped .sum(axis=1)

In [92]:
NYC_grouped_clustering = NYC_restaurant_grouped.drop('Neighborhood', 1)

for n_cluster in range(2, 10):
    kmeans = KMeans(n_clusters=n_cluster).fit(NYC_grouped_clustering)
    label = kmeans.labels_
    sil_coeff = silhouette_score(NYC_grouped_clustering, label, metric='euclidean')
    print("For n_clusters={}, The Silhouette Coefficient is {}".format(n_cluster, sil_coeff))

For n_clusters=2, The Silhouette Coefficient is 0.4831823889458273
For n_clusters=3, The Silhouette Coefficient is 0.36821501790734823
For n_clusters=4, The Silhouette Coefficient is 0.3538229576981859
For n_clusters=5, The Silhouette Coefficient is 0.27641574207431807
For n_clusters=6, The Silhouette Coefficient is 0.2693599828468701
For n_clusters=7, The Silhouette Coefficient is 0.2528847554886187
For n_clusters=8, The Silhouette Coefficient is 0.2452136612036286
For n_clusters=9, The Silhouette Coefficient is 0.21566447223100468


As we can see, n_clusters=2 has highest Silhouette Coefficient. This means that 2 should be the optimal number of clusters.
For n_clusters=2, The Silhouette Coefficient is 0.4070573233323876
Run k-means to cluster the neighborhood into 2 clusters.


In [93]:
# set number of clusters
kclusters = 2

NYC_grouped_clustering = NYC_restaurant_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(NYC_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,

In [94]:
NYC_results = pd.DataFrame(kmeans.cluster_centers_)
NYC_results.columns = NYC_grouped_clustering.columns
NYC_results.index = ['cluster0','cluster1']
NYC_results['Total Sum'] = NYC_results.sum(axis = 1)
NYC_results

,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Brazilian Restaurant,Burmese Restaurant,Cajun / Creole Restaurant,Cambodian Restaurant,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Czech Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Halal Restaurant,Hawaiian Restaurant,Himalayan Restaurant,Hotpot Restaurant,Indian Restaurant,Indonesian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern Greek Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Paella Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Polish Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Romanian Restaurant,Russian Restaurant,Salvadoran Restaurant,Scandinavian Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Soba Restaurant,South American Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Udon Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Total,Total Sum
cluster0,0.010101,0.090909,1.646465,0.151515,1.818182e-01,0.686869,8.080808e-02,0.050505,0.080808,1.010101e-02,0.121212,3.030303e-02,0.101010,1.040404,0.030303,1.717172,0.040404,0.090909,0.242424,3.030303e-02,0.060606,0.161616,0.151515,1.010101e-02,0.090909,2.020202e-02,1.010101e-01,0.151515,0.595960,0.161616,0.727273,0.070707,-8.673617e-19,0.636364,0.070707,0.050505,6.060606e-02,8.080808e-02,1.030303,4.040404e-02,6.060606e-02,2.939394,0.040404,1.303030,3.030303e-02,2.020202e-02,0.989899,0.040404,1.090909,6.060606e-02,0.070707,0.535354,2.000000,0.454545,1.010101e-02,2.020202e-02,5.050505e-02,0.555556,0.020202,0.010101,4.040404e-02,0.333333,0.010101,-8.673617e-19,0.303030,0.787879,-8.673617e-19,0.171717,2.020202e-02,1.010101e-02,0.969697,3.030303e-02,0.111111,2.020202e-02,0.353535,0.363636,0.555556,0.030303,1.343434,1.010101e-02,9.090909e-02,0.030303,0.252525,-1.734723e-18,1.040404,-1.734723e-18,9.090909e-02,0.252525,4.040404e-02,1.010101e-02,0.575758,2.020202e-02,0.494949,29.373737,58.747475
cluster1,0.009852,0.019704,0.778325,0.029557,-7.632783e-17,0.280788,-6.938894e-17,0.009852,0.019704,-8.673617e-18,0.014778,1.734723e-17,0.004926,0.733990,0.009852,1.197044,0.004926,0.014778,0.019704,1.734723e-17,0.024631,0.014778,0.054187,-8.673617e-18,0.004926,-1.734723e-17,4.857226e-17,0.039409,1.073892,0.019704,0.083744,0.034483,4.926108e-03,0.068966,0.019704,0.004926,3.469447e-17,-6.938894e-17,0.261084,-3.469447e-17,3.469447e-17,1.241379,0.009852,0.349754,1.734723e-17,-1.734723e-17,0.039409,0.004926,0.310345,3.469447e-17,0.004926,0.108374,0.610837,0.128079,-8.673617e-18,-1.734723e-17,2.428613e-17,0.113300,0.009852,0.009852,-3.469447e-17,0.093596,0.014778,4.926108e-03,0.009852,0.399015,4.926108e-03,0.019704,-1.734723e-17,-8.673617e-18,0.305419,1.734723e-17,0.004926,-1.734723e-17,0.064039,0.068966,0.349754,0.019704,0.359606,-8.673617e-18,-3.816392e-17,0.004926,0.044335,9.852217e-03,0.162562,9.852217e-03,-3.816392e-17,0.009852,-3.469447e-17,-8.673617e-18,0.073892,-1.734723e-17,0.054187,9.886700,19.773399


In [95]:
NYC_results_merged = pd.DataFrame(NYC_restaurant_grouped['Neighborhood'])

NYC_results_merged['Total'] = NYC_restaurant_grouped['Total']
NYC_results_merged = NYC_results_merged.assign(Cluster_Labels = kmeans.labels_)

In [78]:
print(NYC_results_merged.shape)
NYC_results_merged

(302, 3)


,Neighborhood,Total,Cluster_Labels
0,Allerton,16,0
1,Annadale,3,0
2,Arden Heights,4,0
3,Arlington,3,0
4,Arrochar,4,0
5,Arverne,2,0
6,Astoria,29,1
7,Astoria Heights,11,0
8,Auburndale,34,1
9,Bath Beach,32,1


In [96]:
NYC_merged = NYC_Geo

NYC_merged = NYC_merged.join(NYC_results_merged.set_index('Neighborhood'), on='Neighborhood')

print(NYC_merged.shape)
NYC_merged.head(10)

(306, 6)


,Borough,Neighborhood,Latitude,Longitude,Total,Cluster_Labels
0,Bronx,Wakefield,40.894705,-73.847201,12,1
1,Bronx,Co-op City,40.874294,-73.829939,16,1
2,Bronx,Eastchester,40.887556,-73.827806,12,1
3,Bronx,Fieldston,40.895437,-73.905643,7,1
4,Bronx,Riverdale,40.890834,-73.912585,9,1
5,Bronx,Kingsbridge,40.881687,-73.902818,19,1
6,Manhattan,Marble Hill,40.876551,-73.910660,14,1
7,Bronx,Woodlawn,40.898273,-73.867315,7,1
8,Bronx,Norwood,40.877224,-73.879391,15,1
9,Bronx,Williamsbridge,40.881039,-73.857446,14,1


In [97]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(NYC_merged['Latitude'], NYC_merged['Longitude'], NYC_merged['Neighborhood'], NYC_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [82]:
NYC_merged[NYC_merged['Cluster_Labels'] == 1].reset_index(drop=True)


,Borough,Neighborhood,Latitude,Longitude,Total,Cluster_Labels
0,Bronx,Fordham,40.860997,-73.896427,22,1
1,Bronx,Parkchester,40.837938,-73.856003,28,1
2,Bronx,Belmont,40.857277,-73.888452,22,1
3,Bronx,Unionport,40.829774,-73.850535,20,1
4,Brooklyn,Bay Ridge,40.625801,-74.030621,34,1
5,Brooklyn,Bensonhurst,40.611009,-73.995180,27,1
6,Brooklyn,Sunset Park,40.645103,-74.010316,39,1
7,Brooklyn,Brighton Beach,40.576825,-73.965094,20,1
8,Brooklyn,Sheepshead Bay,40.586890,-73.943186,30,1
9,Brooklyn,Flatbush,40.636326,-73.958401,32,1


In [83]:
NYC_merged[NYC_merged['Total'] == 0].reset_index(drop=True)

,Borough,Neighborhood,Latitude,Longitude,Total,Cluster_Labels
0,Bronx,Clason Point,40.806551,-73.854144,0,0
1,Staten Island,Todt Hill,40.597069,-74.111329,0,0
2,Staten Island,Port Ivory,40.639683,-74.174645,0,0
3,Staten Island,Butler Manor,40.506082,-74.229504,0,0
4,Staten Island,Bloomfield,40.605779,-74.187256,0,0


This clearly shows that Cluster 1 has high number of restaurants located in various neighborhoods of Manhattan, Bronx, Queens, 
Brooklyn and Staten Island.  On the other hand, cluster 0 has no restaurant in Clason Point neighborhood of Bronx and a few 
neighborhoods  of Staten Island. This represents a great opportunity and high potential areas to open new restaurants as there 
is no competition. Meanwhile, restaurants in cluster 1 are likely suffering from intense competition due to oversupply and 
high concentration of restaurants. Therefore, this project recommends developers to capitalize on these findings to 
open new restauarants in neighborhoods in cluster 0 with no competition. Also, developers are advised to avoid 
neighborhoods in cluster 1 which already have high concentration of restaurants and suffering from intense competition.

